# 특강 01: Spotify API 활용 - Client Credentials Flow

## 학습 목표
- Spotify Client Credentials Flow를 구현해보자
- 아티스트/앨범/트랙 검색 기능을 구현해보자
- 음악 데이터 수집 방법을 익혀보자

In [ ]:
# 라이브러리 import
import requests
import base64
import time
import os
from typing import Dict, List, Optional

## 1. Spotify 앱 등록

1. [Spotify for Developers](https://developer.spotify.com/) 접속
2. "Create App" 클릭  
3. **Redirect URI**: `http://localhost:8888/callback` (필수 입력, 실제로는 사용하지 않음)
4. Client ID와 Client Secret 획득

> **참고**: Client Credentials Flow에서는 Redirect URI를 실제로 사용하지 않지만, Spotify 앱 생성 시 필수 항목입니다.

## 2. API 자격 증명 설정 (.env 파일)

.env 파일을 사용하여 안전하게 자격 증명을 관리합니다.

In [10]:
def load_spotify_credentials():
    """.env 파일에서 Spotify API 자격 증명을 로드합니다."""
    env_file = '.env'
    
    # .env 파일이 없으면 템플릿 생성
    if not os.path.exists(env_file):
        template_content = """# Spotify API 자격 증명
SPOTIFY_CLIENT_ID=your_client_id_here
SPOTIFY_CLIENT_SECRET=your_client_secret_here
"""
        with open(env_file, 'w') as f:
            f.write(template_content)
        
        print("📝 .env 파일이 생성되었습니다!")
        print("   실제 Client ID와 Secret을 입력하고 다시 실행하세요")
        return None, None
    
    # .env 파일에서 읽기
    client_id = None
    client_secret = None
    
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                if key == 'SPOTIFY_CLIENT_ID':
                    client_id = value
                elif key == 'SPOTIFY_CLIENT_SECRET':
                    client_secret = value
    
    if client_id == 'your_client_id_here' or client_secret == 'your_client_secret_here':
        print("⚠️ .env 파일에 실제 값을 입력해주세요!")
        return None, None
    
    if client_id and client_secret:
        print("✅ .env 파일에서 자격 증명을 로드했습니다.")
        return client_id, client_secret
    else:
        print("❌ .env 파일에 자격 증명이 없습니다.")
        return None, None

# 자격 증명 로드
CLIENT_ID, CLIENT_SECRET = load_spotify_credentials()

✅ .env 파일에서 자격 증명을 로드했습니다.


## 3. Client Credentials Flow 토큰 발급

OAuth 서버 없이 간단하게 토큰을 발급받습니다.

In [ ]:
class SpotifyClientCredentials:
    def __init__(self, client_id: str, client_secret: str):
        self.client_id = client_id
        self.client_secret = client_secret
        self.access_token = None
        self.token_expires_at = 0
        
    def get_access_token(self) -> Optional[str]:
        """Client Credentials Flow로 액세스 토큰 발급"""
        # 토큰이 유효하면 재사용
        if self.access_token and time.time() < self.token_expires_at:
            return self.access_token
            
        # 새 토큰 발급
        credentials = f"{self.client_id}:{self.client_secret}"
        encoded_credentials = base64.b64encode(credentials.encode()).decode()
        
        headers = {
            'Authorization': f'Basic {encoded_credentials}',
            'Content-Type': 'application/x-www-form-urlencoded'
        }
        
        data = {'grant_type': 'client_credentials'}
        
        response = requests.post('https://accounts.spotify.com/api/token', 
                               headers=headers, data=data)
        
        if response.status_code == 200:
            token_data = response.json()
            self.access_token = token_data['access_token']
            expires_in = token_data.get('expires_in', 3600)
            self.token_expires_at = time.time() + expires_in - 60  # 1분 여유
            
            print("✅ 액세스 토큰을 성공적으로 발급받았습니다!")
            return self.access_token
        else:
            print(f"❌ 토큰 발급 실패: {response.status_code}")
            return None

print("✅ SpotifyClientCredentials 클래스 정의 완료!")

## 4. Spotify API 클라이언트 구현

Client Credentials로 발급받은 토큰으로 Spotify API를 호출하는 클래스입니다.

In [ ]:
class SpotifyAPI:
    def __init__(self, client_credentials: SpotifyClientCredentials):
        self.auth = client_credentials
        self.base_url = "https://api.spotify.com/v1"
    
    def _get_headers(self) -> Optional[Dict]:
        """API 요청 헤더 생성"""
        token = self.auth.get_access_token()
        if not token:
            return None
        return {"Authorization": f"Bearer {token}"}
    
    def search(self, query: str, search_type: str = "track", limit: int = 20) -> Optional[Dict]:
        """음악 검색 (track, artist, album)"""
        headers = self._get_headers()
        if not headers:
            return None
            
        params = {
            "q": query,
            "type": search_type,
            "limit": limit
        }
        
        response = requests.get(f"{self.base_url}/search", 
                               headers=headers, params=params)
        return response.json() if response.status_code == 200 else None
    
    def get_track(self, track_id: str) -> Optional[Dict]:
        """특정 트랙 정보 가져오기"""
        headers = self._get_headers()
        if not headers:
            return None
            
        response = requests.get(f"{self.base_url}/tracks/{track_id}", 
                               headers=headers)
        return response.json() if response.status_code == 200 else None
    
    def get_artist(self, artist_id: str) -> Optional[Dict]:
        """특정 아티스트 정보 가져오기"""
        headers = self._get_headers()
        if not headers:
            return None
            
        response = requests.get(f"{self.base_url}/artists/{artist_id}", 
                               headers=headers)
        return response.json() if response.status_code == 200 else None
    
    def get_artist_top_tracks(self, artist_id: str, market: str = "US") -> Optional[Dict]:
        """아티스트의 인기곡 가져오기"""
        headers = self._get_headers()
        if not headers:
            return None
            
        params = {"market": market}
        response = requests.get(f"{self.base_url}/artists/{artist_id}/top-tracks", 
                               headers=headers, params=params)
        return response.json() if response.status_code == 200 else None

print("✅ SpotifyAPI 클래스 정의 완료!")

## 5. 실제 사용 예시

### 5.1 API 클라이언트 생성

In [13]:
# 자격 증명이 로드되었다면 API 클라이언트 생성
if CLIENT_ID and CLIENT_SECRET:
    # 인증 객체 생성
    auth = SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET)
    
    # API 클라이언트 생성
    spotify = SpotifyAPI(auth)
    
    print("✅ Spotify API 클라이언트가 생성되었습니다.")
    print("   다음 셀에서 API를 사용해보세요.")
else:
    print("❌ 먼저 .env 파일에 자격 증명을 입력하세요!")

✅ Spotify API 클라이언트가 생성되었습니다.
   다음 셀에서 API를 사용해보세요.


### 5.2 음악 검색하기

In [15]:
# 1. 트랙 검색
search_results = spotify.search("NewJeans Get Up", "track", limit=5)

if search_results and 'tracks' in search_results:
    print("=== '뉴진스 Get Up' 검색 결과 ===")
    for i, track in enumerate(search_results['tracks']['items'], 1):
        artists = ", ".join([artist['name'] for artist in track['artists']])
        print(f"{i}. {track['name']} - {artists}")
        print(f"   앨범: {track['album']['name']}")
        print(f"   발매일: {track['album']['release_date']}")
        print(f"   인기도: {track['popularity']}/100")
        print(f"   ID: {track['id']}")
        print()
else:
    print("검색 결과가 없거나 API 호출에 실패했습니다.")

=== '뉴진스 Get Up' 검색 결과 ===
1. Get Up - NewJeans
   앨범: NewJeans 2nd EP 'Get Up'
   발매일: 2023-07-21
   인기도: 68/100
   ID: 1wUnuiXMMvhudmzvcCtlZP

2. NEW DROP - Don Toliver
   앨범: Trending Edit Audios
   발매일: 2025-08-19
   인기도: 0/100
   ID: 0dNLRYTbYGPMwNIZCby7cV

3. ETA - NewJeans
   앨범: NewJeans 2nd EP 'Get Up'
   발매일: 2023-07-21
   인기도: 77/100
   ID: 56v8WEnGzLByGsDAXDiv4d

4. NEW DROP - Don Toliver
   앨범: Viral USA - Trending Now
   발매일: 2025-08-27
   인기도: 0/100
   ID: 4DAeFo5Sh4lxZhBYFW1NU7

5. How Sweet - NewJeans
   앨범: How Sweet
   발매일: 2024-05-24
   인기도: 76/100
   ID: 38tXZcL1gZRfbqfOG0VMTH



### 5.3 아티스트 정보 및 인기곡

In [ ]:
# 아티스트 검색 및 정보
artist_search = spotify.search("NewJeans", "artist", limit=1)

if artist_search and 'artists' in artist_search and artist_search['artists']['items']:
    artist = artist_search['artists']['items'][0]
    artist_id = artist['id']
    
    print(f"=== {artist['name']} 아티스트 정보 ===")
    print(f"장르: {', '.join(artist['genres']) if artist['genres'] else '정보 없음'}")
    print(f"팔로워: {artist['followers']['total']:,}명")
    print(f"인기도: {artist['popularity']}/100")
    print()
    
    # 아티스트의 인기곡 가져오기
    top_tracks = spotify.get_artist_top_tracks(artist_id)
    
    if top_tracks and 'tracks' in top_tracks:
        print(f"=== {artist['name']} 인기곡 TOP 10 ===")
        for i, track in enumerate(top_tracks['tracks'], 1):
            print(f"{i:2d}. {track['name']}")
            print(f"    앨범: {track['album']['name']} ({track['album']['release_date'][:4]})")
            print(f"    인기도: {track['popularity']}/100")
            duration_sec = track['duration_ms'] // 1000
            print(f"    길이: {duration_sec // 60}:{duration_sec % 60:02d}")
            print()
else:
    print("아티스트를 찾을 수 없습니다.")

## 6. 주의사항 및 베스트 프랙티스

### 6.1 보안 관련
- **절대로 Client Secret을 공개하지 마세요**
- `.env` 파일을 사용하여 자격 증명을 관리하세요
- `.env` 파일이 `.gitignore`에 포함되어 있는지 확인하세요

### 6.2 API 사용량 관리
- Spotify API는 Rate Limiting이 있음 (일반적으로 초당 100회 요청)
- 요청 간 적절한 지연 시간 추가 (`time.sleep(0.1)`)
- 대량 데이터는 배치로 처리

### 6.3 현재 사용 가능한 Spotify API 기능
- ✅ 트랙/아티스트/앨범 검색
- ✅ 트랙 기본 정보 (이름, 아티스트, 앨범, 발매일, 인기도)
- ✅ 아티스트 정보 (장르, 팔로워, 인기도)
- ✅ 아티스트 인기곡

### 6.4 토큰 관리
- Access Token은 1시간 후 만료
- 자동으로 새 토큰 발급 로직이 구현되어 있음
- 토큰은 메모리에만 저장 (보안상 안전)